In [2]:
import numpy as np
from math import pi
import datetime as dt
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, LogAxis, Range1d
from scipy.fftpack import fft, fftfreq, fftshift, rfft
output_file('Fourier.html')

In [3]:
tick = pd.read_csv('RIM6_4.trd', usecols=(0, 1, 2, 3, 4, 5, 6),
                   parse_dates={'DateTime': [0,1]}, index_col=False)

In [4]:
tick = tick.rename(columns={'<LAST>': 'Price', '<VOL>': 'Vol', '<DIRECTION>': 'Dir'})

In [5]:
tick.ix[tick.Dir=='Sell', 'VolSell'] = tick.Vol

In [6]:
tick.ix[tick.Dir=='Buy', 'VolBuy'] = tick.Vol

In [7]:
Nticks=100 #Количество тиков для бара и ресэмплинг

In [8]:
# main=pd.DataFrame(index=np.arange(0, (len(tr.index)/Nticks+Nticks)),
#                 columns=('DateTime', 'open', 'high', 'low', 'close', 'vBuy', 'vSell'))
rows_list=[]
for i in range(0, len(tick.index), Nticks):
    dict1={}
#    srez = tr.iloc[i:i+Nticks]
    op=tick.Price.iloc[i]
    hi=tick.Price.iloc[i:i+Nticks].max()
    lo=tick.Price.iloc[i:i+Nticks].min()
    if i+Nticks<len(tick.index):
        cl=tick.loc[i+Nticks-1].Price
    else:
        cl=tick.loc[len(tick.index)-1].Price
    
    vBuy=tick.VolBuy.iloc[i:i+Nticks].sum()
    vSell=tick.VolSell.iloc[i:i+Nticks].sum()
    nBuy=tick.VolBuy.iloc[i:i+Nticks].count()
    nSell=tick.VolSell.iloc[i:i+Nticks].count()
    tClose=tick.DateTime.iloc[i:i+Nticks].max()
    dtime=tick.DateTime.iloc[i:i+Nticks].max()-tick.DateTime.iloc[i:i+Nticks].min()
    dict1.update({'DateTime': tClose, 'open': op, 'high': hi,
                 'low': lo, 'close': cl, 'TickBuy': nBuy,
                'VolBuy': vBuy, 'TickSell': nSell, 'VolSell': vSell, 'DeltaTime': dtime})
    rows_list.append(dict1)
    
res=pd.DataFrame(rows_list)
res['Dsec']=res['DeltaTime'].dt.total_seconds()
res['TSpeedBuy']=res['TickBuy']/res['Dsec']
res['TSpeedSell']=res['TickSell']/res['Dsec']
res['VSpeedBuy']=res['VolBuy']/res['Dsec']
res['VSpeedSell']=res['VolSell']/res['Dsec']
res['OTO']=((res['TickBuy']-res['TickSell'])/(res['TickBuy']+res['TickSell'])*0.65
            + (res['VolBuy']-res['VolSell'])/(res['VolBuy']+res['VolSell'])*0.35)*100.0
res['OTON']=((res['TickBuy']*res['VolBuy']-res['TickSell']*res['VolSell'])/
             (res['TickBuy']*res['VolBuy']+res['TickSell']*res['VolSell']))*100.0
res['Delta']=res['VolBuy']-res['VolSell']
res['CumDelta']=res['Delta'].cumsum()

In [9]:
res

,DateTime,DeltaTime,TickBuy,TickSell,VolBuy,VolSell,close,high,low,open,Dsec,TSpeedBuy,TSpeedSell,VSpeedBuy,VSpeedSell,OTO,OTON,Delta,CumDelta
0,2016-04-20 10:00:00,00:00:00,0,100,NaN,349.0,91090,91860,91090,91860,0.0,NaN,inf,NaN,inf,NaN,NaN,NaN,NaN
1,2016-04-20 10:00:01,00:00:01,4,96,16.0,564.0,91000,91100,91000,91090,1.0,4.000000,96.000000,16.000000,564.000000,-92.868966,-99.763872,-548.0,-548.0
2,2016-04-20 10:00:01,00:00:00,7,93,11.0,347.0,90800,91000,90800,91000,0.0,inf,inf,inf,inf,-88.749162,-99.523927,-336.0,-884.0
3,2016-04-20 10:00:01,00:00:00,34,66,48.0,143.0,90750,91000,90750,90800,0.0,inf,inf,inf,inf,-38.208377,-70.514905,-95.0,-979.0
4,2016-04-20 10:00:01,00:00:00,51,49,79.0,132.0,90950,90990,90730,90960,0.0,inf,inf,inf,inf,-7.491469,-23.235210,-53.0,-1032.0
5,2016-04-20 10:00:01,00:00:00,29,71,34.0,113.0,90850,90920,90700,90900,0.0,inf,inf,inf,inf,-46.109524,-78.110778,-79.0,-1111.0
6,2016-04-20 10:00:01,00:00:00,49,51,70.0,104.0,90880,90880,90700,90850,0.0,inf,inf,inf,inf,-8.139080,-21.456377,-34.0,-1145.0
7,2016-04-20 10:00:01,00:00:00,17,83,18.0,138.0,90690,90880,90650,90880,0.0,inf,inf,inf,inf,-69.823077,-94.795918,-120.0,-1265.0
8,2016-04-20 10:00:01,00:00:00,42,58,57.0,98.0,90750,90810,90660,90700,0.0,inf,inf,inf,inf,-19.658065,-40.727903,-41.0,-1306.0
9,2016-04-20 10:00:02,00:00:01,58,42,108.0,57.0,90820,90830,90740,90750,1.0,58.000000,42.000000,108.000000,57.000000,21.218182,44.698545,51.0,-1255.0


In [13]:
# res = res.set_index('DateTime')
mids = (res.open + res.close)/2
spans = abs(res.close - res.open)
# print res, mids, spans
inc = res.close > res.open
dec = res.open > res.close
#w = 12*60*60
w = 0.5
# print res.index[inc] , mids, spans, inc, dec
resfft = rfft(res[-128:].OTON)
resfftfreq = fftfreq(len(resfft))

In [12]:
p = figure(x_range=(0, len(res.index)), plot_width=800, plot_height=600, title='RIM6 CandleStick')
p.y_range=Range1d(start=90500, end=95000)
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.3
p.segment(res.index.values, res.high, res.index.values, res.low, color='black')
p.rect(res.index[inc], mids[inc], w, spans[inc], fill_color='#6CF350', line_color='green')
p.rect(res.index[dec], mids[dec], w, spans[dec], fill_color='#FF0000', line_color='red')
p.extra_y_ranges = {}
p.extra_y_ranges['TickSpeed'] = Range1d(start=0, end=100)
p.extra_y_ranges['OTO'] = Range1d(start=-60, end=60)
p.extra_y_ranges['OTON'] = Range1d(start=-60, end=60)
p.extra_y_ranges['CumDelta'] = Range1d(start=-3000, end=52000)
#p.line(res.index, res.TSpeedBuy, color='blue', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedBuy.rolling(window=4).median(), color='green', y_range_name='TickSpeed', alpha=0.8)
#p.line(res.index, res.TSpeedSell, color='red', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedSell.rolling(window=9).median(), color='red', y_range_name='TickSpeed', alpha=0.8)
p.line(res.index, res.OTO.rolling(window=15).mean(), color='brown', y_range_name='OTO', alpha=0.9)
p.line(res.index, res.OTON.rolling(window=15).mean(), color='blue', y_range_name='OTON', alpha=0.9)
p.line(res.index, res.CumDelta, color='black', y_range_name='CumDelta', alpha=0.9)
p.add_layout(LogAxis(y_range_name='TickSpeed'), 'left')
p.add_layout(LinearAxis(y_range_name='OTO'), 'left')
p.add_layout(LinearAxis(y_range_name='OTON'), 'left')
p.add_layout(LinearAxis(y_range_name='CumDelta'), 'left')
#show(p)
l=figure(plot_width=800, plot_height=600, title='FFT')
l.line(resfftfreq, resfft)
m = gridplot([[p, l]])

show(m)